In [2]:
from neo4j import GraphDatabase
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
driver=GraphDatabase.driver(uri=os.getenv('NEO4J_URI_ONLINE'),auth=(os.getenv('NEO4J_USERNAME_ONLINE'),os.getenv('NEO4J_PASSWORD_ONLINE')))

In [10]:
import json 
with open('pdf_files/neo4j_nodes&edges.json','r') as file:
    results=json.load(file)
len(results)

269

In [11]:
import re 
existing_set=[]

def sanitize_label(label):
    sanitized=re.sub(r"[^\w]", "_", label)
    return re.sub(r"_+", "_", sanitized).strip("_")

def insert_entities(session,entities):
    for entity in entities:
        
        sanitize_labels=sanitize_label(entity['id'])
        query=f"""
            MERGE (n:{sanitize_labels} {{id:$id}})
            ON CREATE SET
                n.role=$role,
                n.description=$description
            ON MATCH SET
                n.role=$role,
                n.description=$description

            """
        session.run(query,label=entity['label'], id=entity['id'],role=entity['role'],description=entity['label'])
        print(f'Inserted entity: {entity["id"]}')
        existing_set.append(entity['label'])
    
def insert_relationship(session,relationships):
    for relation in relationships:
        query = f"""
        MATCH (a {{id: $source}}), (b {{id: $target}})
        MERGE (a)-[:{sanitize_label(relation['type'])} {{
            id: $id,
            description: $description
        }}]->(b)
        """
        session.run(query,source=relation['source'],target=relation['target'],id=relation['id'],description=relation['description'])
        print(f'Inserted relationship: {relation["id"]}')


In [12]:
with driver.session() as session:
    for result in results:
        try:
            session.execute_write(insert_entities,result['entities'])
            session.execute_write(insert_relationship,result['relationships'])
        except Exception as e:
            print(f'Error {e}')
driver.close()

Inserted entity: Elon Musk
Inserted entity: South Africa
Inserted entity: Canada
Inserted entity: United States
Error 'relationships'
Inserted entity: Elon Musk
Inserted entity: SpaceX
Inserted entity: Tesla Inc.
Inserted entity: X Corp
Inserted entity: The Boring Company
Inserted entity: xAI
Inserted entity: Neuralink
Inserted entity: OpenAI
Inserted entity: University of Pennsylvania
Inserted entity: Zip2
Inserted entity: PayPal
Inserted entity: Compaq
Inserted entity: eBay
Inserted relationship: relationship1
Inserted relationship: relationship2
Inserted relationship: relationship3
Inserted relationship: relationship4
Inserted relationship: relationship5
Inserted relationship: relationship6
Inserted relationship: relationship7
Inserted relationship: relationship8
Inserted relationship: relationship9
Inserted relationship: relationship10
Inserted relationship: relationship11
Inserted relationship: relationship12
Inserted entity: Elon Musk
Inserted entity: SolarCity
Inserted entity: T